# 🧬 Módulo 2: Representación y Visualización Molecular
## Actividad 2.1: Formatos de Archivos Moleculares

<div align="center">
  
**Universidad de Caldas - Departamento de Química**  
*Introducción a la Química Computacional (173G7G)*  
**Profesor:** José Mauricio Rodas Rodríguez

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maurorodas/Quimica_computacional_173G7G/blob/main/modulo_02_mecanica_molecular/01_formatos_moleculares.ipynb)

</div>

---

## 🎯 Objetivos de Aprendizaje

Al finalizar esta actividad, serás capaz de:
- Comprender los formatos de archivos moleculares más comunes
- Leer y escribir archivos XYZ, PDB, MOL/MOL2, y SDF
- Convertir entre diferentes formatos usando RDKit
- Extraer información estructural de archivos moleculares
- Seleccionar el formato apropiado para cada aplicación
- Trabajar con colecciones de moléculas en archivos SDF

---

## 📚 ¿Por qué Existen Tantos Formatos?

### El Problema de la Representación Molecular

> "Cada programa de química tiene su formato favorito... y ninguno es perfecto"

Los formatos de archivos moleculares surgieron para resolver diferentes necesidades:

- 🔬 **Cristalografía**: CIF, PDB → Datos experimentales
- 💊 **Diseño de fármacos**: MOL2, SDF → Propiedades y descriptores
- ⚛️ **Química cuántica**: XYZ, Gaussian → Geometrías para cálculos
- 🧬 **Bioquímica**: PDB, mmCIF → Macromoléculas
- 📊 **Bases de datos**: SDF, SMILES → Almacenamiento eficiente

### Características Comunes

Todos los formatos contienen, al menos:
- ✅ **Identidad atómica** - Qué átomos están presentes
- ✅ **Coordenadas** - Dónde están en el espacio (2D o 3D)

Algunos formatos adicionales incluyen:
- ⚡ **Conectividad** - Quién está enlazado con quién
- 🔋 **Cargas parciales** - Distribución electrónica
- 🏷️ **Propiedades** - Energía, descriptores, anotaciones
- 📐 **Información cristalográfica** - Celdas, simetría

---

## 📦 Instalación e Importación

In [ ]:
# Instalación en Google Colab
import sys
if 'google.colab' in sys.modules:
    !pip install rdkit -q
    print("✓ RDKit instalado")

# Importar bibliotecas
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Draw
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

print(f"✓ RDKit versión: {Chem.rdBase.rdkitVersion}")
print("✓ Bibliotecas importadas correctamente")

---

## 📄 Formato XYZ - El Más Simple

### Características

- ✅ **Simplicidad**: Solo coordenadas atómicas
- ✅ **Legibilidad**: Fácil de leer para humanos
- ❌ **Sin conectividad**: No especifica enlaces
- ❌ **Sin propiedades**: Solo geometría

### Estructura del Archivo

```
número_de_átomos
comentario o título
símbolo_atómico  x  y  z
símbolo_atómico  x  y  z
...
```

### Ejemplo: Molécula de Agua

In [ ]:
# Crear archivo XYZ de agua
xyz_agua = """3
Molécula de agua (H2O)
O    0.000000    0.000000    0.119262
H    0.000000    0.763239   -0.477047
H    0.000000   -0.763239   -0.477047
"""

# Guardar archivo
with open('agua.xyz', 'w') as f:
    f.write(xyz_agua)

print("Contenido del archivo agua.xyz:")
print(xyz_agua)
print("✓ Archivo guardado")

### Leer Archivo XYZ

In [ ]:
# Función para leer XYZ
def leer_xyz(filename):
    """
    Lee un archivo XYZ y extrae información atómica.
    """
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    n_atomos = int(lines[0].strip())
    titulo = lines[1].strip()
    
    atomos = []
    coordenadas = []
    
    for line in lines[2:2+n_atomos]:
        parts = line.split()
        atomos.append(parts[0])
        coordenadas.append([float(x) for x in parts[1:4]])
    
    return {
        'n_atomos': n_atomos,
        'titulo': titulo,
        'atomos': atomos,
        'coordenadas': np.array(coordenadas)
    }

# Leer el archivo
mol_data = leer_xyz('agua.xyz')

print(f"Título: {mol_data['titulo']}")
print(f"Número de átomos: {mol_data['n_atomos']}")
print("\nÁtomos y coordenadas (Å):")
for atomo, coord in zip(mol_data['atomos'], mol_data['coordenadas']):
    print(f"{atomo:>2s}  {coord[0]:10.6f}  {coord[1]:10.6f}  {coord[2]:10.6f}")

---

## 🧬 Formato PDB - Protein Data Bank

### Características

- ✅ **Estándar para biomoléculas**: Proteínas, ADN, ARN
- ✅ **Información rica**: Residuos, cadenas, factores B
- ✅ **Datos experimentales**: Resolución, método
- ⚠️ **Formato de columnas fijas**: Requiere formato preciso

### Estructura de la Línea ATOM

```
ATOM   [serial] [name] [resName] [chainID] [resSeq]    [x]     [y]     [z]  [occ] [tempF] [element]
```

**Explicación de cada columna:**

| Campo | Columnas | Descripción | Ejemplo |
|-------|----------|-------------|---------|
| **ATOM** | 1-6 | Identificador de registro | ATOM |
| **serial** | 7-11 | Número de átomo (secuencial) | 1, 2, 3... |
| **name** | 13-16 | Nombre del átomo | CA, CB, N, O |
| **resName** | 18-20 | Nombre del residuo (3 letras) | ALA, GLY, TYR |
| **chainID** | 22 | Identificador de cadena | A, B, C |
| **resSeq** | 23-26 | Número del residuo en la secuencia | 1, 2, 100 |
| **x, y, z** | 31-38, 39-46, 47-54 | Coordenadas cartesianas (Å) | 12.345, -5.678 |
| **occ** | 55-60 | Ocupancia (0.0-1.0) | 1.00 |
| **tempF** | 61-66 | Factor de temperatura (B-factor, Ų) | 25.50 |
| **element** | 77-78 | Símbolo del elemento | C, N, O, FE |

**Ejemplo de línea real:**
```
ATOM      1  N   MET A   1      27.340  24.430   2.614  1.00  9.67           N
```
- Átomo #1, nitrógeno (N) del residuo metionina (MET), cadena A, residuo #1
- Coordenadas: x=27.340, y=24.430, z=2.614 Å
- Ocupancia completa (1.00), factor B=9.67 Ų

### Ejemplo: Descargar Proteína desde PDB

In [ ]:
# Descargar estructura de aspirina (ID: 2OYE)
import urllib.request

pdb_id = "2OYE"  # Complejo aspirina-COX
url = f"https://files.rcsb.org/download/{pdb_id}.pdb"

try:
    urllib.request.urlretrieve(url, f"{pdb_id}.pdb")
    print(f"✓ Estructura {pdb_id} descargada")
    
    # Leer con RDKit
    mol_pdb = Chem.MolFromPDBFile(f"{pdb_id}.pdb", sanitize=False, removeHs=False)
    
    if mol_pdb:
        print(f"  Átomos totales: {mol_pdb.GetNumAtoms()}")
        
        # Leer información del header
        with open(f"{pdb_id}.pdb", 'r') as f:
            for i, line in enumerate(f):
                if i > 10:  # Solo primeras líneas
                    break
                if line.startswith(('HEADER', 'TITLE', 'COMPND')):
                    print(f"  {line.strip()}")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Verifica tu conexión a internet")

---

## 🧪 Formato MOL/MOL2 - Moléculas Orgánicas

### Características del Formato MOL

- ✅ **Conectividad explícita**: Especifica todos los enlaces
- ✅ **Tipos de enlace**: Simple, doble, triple, aromático
- ✅ **Estereoquímica**: Configuración 3D
- ✅ **Ampliamente soportado**: Mayoría de software químico

### Ejemplo: Crear y Guardar Molécula

In [ ]:
# Crear cafeína desde SMILES
cafeina = Chem.MolFromSmiles('CN1C=NC2=C1C(=O)N(C(=O)N2C)C')

# Agregar coordenadas 2D
AllChem.Compute2DCoords(cafeina)

# Guardar como MOL
with Chem.SDWriter('cafeina.mol') as writer:
    writer.write(cafeina)

print("✓ Archivo cafeina.mol creado")
print(f"  Fórmula: {Chem.rdMolDescriptors.CalcMolFormula(cafeina)}")
print(f"  Masa molecular: {Descriptors.MolWt(cafeina):.2f} g/mol")
print(f"  Átomos: {cafeina.GetNumAtoms()}")
print(f"  Enlaces: {cafeina.GetNumBonds()}")

# Visualizar
img = Draw.MolToImage(cafeina, size=(300, 300))
display(img)

---

## 📚 Formato SDF - Structure Data File

### Características

- ✅ **Múltiples moléculas**: Un archivo, muchas estructuras
- ✅ **Propiedades asociadas**: Datos experimentales, descriptores
- ✅ **Formato de bases de datos**: PubChem, ChEMBL
- ✅ **Búsquedas eficientes**: Screening virtual

### Ejemplo: Crear Biblioteca de Moléculas

In [ ]:
# Crear una pequeña biblioteca de analgésicos
moleculas = {
    'Aspirina': 'CC(=O)Oc1ccccc1C(=O)O',
    'Paracetamol': 'CC(=O)Nc1ccc(O)cc1',
    'Ibuprofeno': 'CC(C)Cc1ccc(cc1)C(C)C(=O)O',
    'Naproxeno': 'COc1ccc2cc(ccc2c1)C(C)C(=O)O'
}

# Crear archivo SDF
writer = Chem.SDWriter('analgesicos.sdf')

for nombre, smiles in moleculas.items():
    mol = Chem.MolFromSmiles(smiles)
    AllChem.Compute2DCoords(mol)
    
    # Agregar propiedades
    mol.SetProp('_Name', nombre)
    mol.SetProp('SMILES', smiles)
    mol.SetProp('MW', f"{Descriptors.MolWt(mol):.2f}")
    mol.SetProp('LogP', f"{Descriptors.MolLogP(mol):.2f}")
    mol.SetProp('HBD', str(Descriptors.NumHDonors(mol)))
    mol.SetProp('HBA', str(Descriptors.NumHAcceptors(mol)))
    
    writer.write(mol)

writer.close()
print("✓ Archivo analgesicos.sdf creado")
print(f"  Moléculas: {len(moleculas)}")

# Visualizar todas
img = Draw.MolsToGridImage([Chem.MolFromSmiles(s) for s in moleculas.values()], 
                           legends=list(moleculas.keys()),
                           molsPerRow=2, subImgSize=(250, 250))
display(img)

### 📖 Leer y Analizar SDF

Ahora leemos el archivo SDF y extraemos todas las moléculas con sus propiedades:

In [ ]:
# Leer archivo SDF
suppl = Chem.SDMolSupplier('analgesicos.sdf')

print("=" * 70)
print("ANÁLISIS DE BIBLIOTECA MOLECULAR")
print("=" * 70)

datos = []
for i, mol in enumerate(suppl, 1):
    if mol is None:
        print(f"⚠ Molécula {i}: Error al leer")
        continue
    
    # Extraer propiedades
    nombre = mol.GetProp('_Name')
    smiles = mol.GetProp('SMILES')
    mw = mol.GetProp('MW')
    logp = mol.GetProp('LogP')
    hbd = mol.GetProp('HBD')
    hba = mol.GetProp('HBA')
    
    print(f"\n{i}. {nombre}")
    print(f"   SMILES: {smiles}")
    print(f"   Peso molecular: {mw} g/mol")
    print(f"   LogP: {logp}")
    print(f"   Donadores H: {hbd}")
    print(f"   Aceptores H: {hba}")
    
    datos.append({
        'Nombre': nombre,
        'SMILES': smiles,
        'MW': float(mw),
        'LogP': float(logp),
        'HBD': int(hbd),
        'HBA': int(hba)
    })

# Crear tabla resumen
df = pd.DataFrame(datos)
print("\n" + "=" * 70)
print("TABLA RESUMEN")
print("=" * 70)
print(df.to_string(index=False))

---

## 🔄 Conversión entre Formatos

RDKit permite convertir entre diferentes formatos de archivo. Veamos ejemplos prácticos:

### De SMILES a múltiples formatos

In [ ]:
# Molécula de ejemplo: Ácido acetilsalicílico (Aspirina)
smiles = 'CC(=O)Oc1ccccc1C(=O)O'
mol = Chem.MolFromSmiles(smiles)
mol.SetProp('_Name', 'Aspirina')

# Agregar hidrógenos explícitos (importante para algunos formatos)
mol_h = Chem.AddHs(mol)

# Generar coordenadas 3D
AllChem.EmbedMolecule(mol_h, randomSeed=42)
AllChem.MMFFOptimizeMolecule(mol_h)

print("CONVERSIÓN DE FORMATOS DESDE SMILES")
print("=" * 60)

# 1. A formato XYZ
xyz_block = Chem.MolToXYZBlock(mol_h)
with open('aspirina.xyz', 'w') as f:
    f.write(xyz_block)
print("✓ aspirina.xyz creado")

# 2. A formato MOL
Chem.MolToMolFile(mol_h, 'aspirina.mol')
print("✓ aspirina.mol creado")

# 3. A formato PDB
Chem.MolToPDBFile(mol_h, 'aspirina.pdb')
print("✓ aspirina.pdb creado")

# 4. Mostrar el bloque MOL
print("\n" + "-" * 60)
print("CONTENIDO DEL ARCHIVO MOL (primeras líneas):")
print("-" * 60)
mol_block = Chem.MolToMolBlock(mol_h)
print('\n'.join(mol_block.split('\n')[:15]) + '\n...')

# Visualizar la estructura 3D
from IPython.display import Image
img = Draw.MolToImage(mol, size=(300, 300))
display(img)

### Conversión entre archivos existentes

In [ ]:
# Leer un SDF y convertir todas las moléculas a MOL individuales
suppl = Chem.SDMolSupplier('analgesicos.sdf')

print("CONVERSIÓN SDF → MOL individuales")
print("=" * 60)

for i, mol in enumerate(suppl, 1):
    if mol is None:
        continue
    
    nombre = mol.GetProp('_Name')
    archivo = f'{nombre.lower()}.mol'
    
    # Guardar cada molécula
    Chem.MolToMolFile(mol, archivo)
    print(f"✓ {archivo} creado")

print(f"\n{i} archivos MOL creados")

# Leer un XYZ y agregar conectividad (convertir a MOL)
print("\n" + "=" * 60)
print("CONVERSIÓN XYZ → MOL (agregando conectividad)")
print("=" * 60)

# Leer XYZ
with open('water.xyz', 'r') as f:
    xyz_lines = f.readlines()

# Parsear manualmente y crear molécula
mol = Chem.RWMol()
for line in xyz_lines[2:]:  # Saltar header
    parts = line.split()
    if len(parts) == 4:
        atom_symbol = parts[0]
        atom = Chem.Atom(atom_symbol)
        mol.AddAtom(atom)

# Agregar enlaces (H2O: O-H, O-H)
mol.AddBond(0, 1, Chem.BondType.SINGLE)  # O-H
mol.AddBond(0, 2, Chem.BondType.SINGLE)  # O-H

# Convertir a molécula inmutable
mol = mol.GetMol()

# Generar coordenadas 2D
AllChem.Compute2DCoords(mol)

# Guardar como MOL
Chem.MolToMolFile(mol, 'water.mol')
print("✓ water.mol creado con conectividad")

---

## 📊 Tabla Comparativa de Formatos

| **Formato** | **Extensión** | **Coordenadas** | **Conectividad** | **Propiedades** | **Múltiples Mols** | **Uso Principal** |
|-------------|---------------|-----------------|------------------|-----------------|-------------------|-------------------|
| **XYZ** | .xyz | ✅ 3D | ❌ | ❌ | ❌ | Geometrías optimizadas, cálculos QM |
| **PDB** | .pdb | ✅ 3D | ⚠️ Implícita | ✅ | ✅ (modelos) | Biomoléculas, proteínas, cristalografía |
| **MOL** | .mol, .sdf | ✅ 2D/3D | ✅ Explícita | ⚠️ Limitadas | ❌ | Moléculas orgánicas pequeñas |
| **MOL2** | .mol2 | ✅ 3D | ✅ Explícita | ✅ | ❌ | Docking, modelado molecular |
| **SDF** | .sdf | ✅ 2D/3D | ✅ Explícita | ✅ Extensas | ✅ | Bases de datos, screening virtual |

**Leyenda:**
- ✅ = Soporta completamente
- ⚠️ = Soporta parcialmente
- ❌ = No soporta

---

## 🎯 Ejercicios Prácticos

### **Ejercicio 1: Básico - Crear y Leer**

Crea un archivo XYZ para el dióxido de carbono (CO₂) con las siguientes coordenadas aproximadas:
- C: (0.0, 0.0, 0.0)
- O: (1.16, 0.0, 0.0)
- O: (-1.16, 0.0, 0.0)

Luego lee el archivo y calcula:
1. La distancia entre cada oxígeno y el carbono
2. El ángulo O-C-O

<details>
<summary>💡 Pista</summary>
Usa NumPy para calcular distancias: `np.linalg.norm(coord1 - coord2)`
Para el ángulo, usa el producto punto: `np.arccos(dot_product / (norm1 * norm2))`
</details>

### **Ejercicio 2: Intermedio - Conversión con Propiedades**

Descarga 5 moléculas de tu elección desde PubChem (puedes usar SMILES) y:
1. Créalas con RDKit
2. Genera coordenadas 3D
3. Calcula propiedades: Peso molecular, LogP, número de átomos
4. Guárdalas en un archivo SDF con todas las propiedades
5. Lee el archivo y crea una tabla comparativa con Pandas

<details>
<summary>💡 Pista</summary>
Para descargar de PubChem por nombre:
```python
from rdkit.Chem import PandasTools
import requests
```
Usa `Descriptors` de RDKit para calcular propiedades.
</details>

### **Ejercicio 3: Avanzado - Pipeline de Procesamiento**

Crea un script que:
1. Lee un archivo SDF con múltiples moléculas
2. Filtra moléculas según criterios de Lipinski (MW < 500, LogP < 5, HBD ≤ 5, HBA ≤ 10)
3. Para las moléculas que pasan el filtro:
   - Genera 10 confórmeros
   - Optimiza geometrías
   - Calcula energías
4. Guarda los confórmeros de menor energía en archivos separados (MOL)
5. Crea un reporte en formato texto con estadísticas

<details>
<summary>💡 Pista</summary>
Usa `AllChem.EmbedMultipleConfs()` para generar confórmeros.
Para energías: `AllChem.MMFFOptimizeMolecule()` retorna la energía.
Regla de Lipinski: filtros para fármacos orales.
</details>

---

## 📚 Recursos Adicionales

### Documentación Oficial
- **RDKit**: [rdkit.org/docs](https://www.rdkit.org/docs/)
  - [Guía de lectura/escritura de moléculas](https://www.rdkit.org/docs/GettingStartedInPython.html#reading-and-writing-molecules)
  - [Descriptores moleculares](https://www.rdkit.org/docs/GettingStartedInPython.html#molecular-descriptors)
  
- **OpenBabel**: [openbabel.org](https://openbabel.org/)
  - Soporta 110+ formatos químicos
  - Excelente para conversiones batch

### Bases de Datos Moleculares
- **PubChem**: [pubchem.ncbi.nlm.nih.gov](https://pubchem.ncbi.nlm.nih.gov/) - Moléculas pequeñas
- **RCSB PDB**: [rcsb.org](https://www.rcsb.org/) - Estructuras de proteínas
- **ChEMBL**: [ebi.ac.uk/chembl](https://www.ebi.ac.uk/chembl/) - Bioactividad
- **ZINC**: [zinc.docking.org](https://zinc.docking.org/) - Compuestos comerciales

### Especificaciones de Formatos
- [XYZ Format Specification](http://openbabel.org/wiki/XYZ)
- [PDB Format Guide v3.30](https://www.wwpdb.org/documentation/file-format)
- [MDL MOL/SDF Format](http://c4.cabrillo.edu/404/ctfile.pdf)

### Tutoriales Recomendados
- 📖 [RDKit Cookbook](https://www.rdkit.org/docs/Cookbook.html)
- 🎥 [Cheminformatics Tutorial (YouTube)](https://www.youtube.com/results?search_query=rdkit+tutorial)
- 💻 [Greg Landrum's RDKit Blog](https://greglandrum.github.io/rdkit-blog/)

---

## 🎓 Resumen

En esta actividad aprendiste:

✅ **Formatos básicos**:
- **XYZ**: Coordenadas atómicas simples, ideal para geometrías optimizadas
- **PDB**: Estándar para biomoléculas con información de residuos
- **MOL/MOL2**: Moléculas orgánicas con conectividad explícita
- **SDF**: Bases de datos con múltiples moléculas y propiedades extensas

✅ **Habilidades desarrolladas**:
- Lectura y escritura de diferentes formatos
- Conversión entre formatos preservando información
- Extracción de propiedades moleculares
- Manejo de bibliotecas de compuestos

✅ **Herramientas dominadas**:
- RDKit para manipulación molecular
- NumPy/Pandas para análisis de datos
- Visualización de estructuras 2D/3D

### 🚀 Próximos Pasos
En la siguiente actividad exploraremos **notaciones químicas lineales** (SMILES, InChI, SMARTS) y cómo utilizarlas para búsquedas y generación de moléculas.

---

<div align="center">

<div align="center">

## 🎉 ¡Felicitaciones!

Has completado la **Actividad 2.1: Formatos de Archivos Moleculares**

**Siguiente actividad**: Notaciones Químicas (SMILES, InChI, SMARTS)

[![Inicio](https://img.shields.io/badge/⬅️_Actividad_1.8-Entornos_Virtuales-blue.svg)](../modulo_01_fundamentos/08_entornos_virtuales.ipynb)
[![Next](https://img.shields.io/badge/Actividad_2.2_➡️-Notaciones_Químicas-green.svg)](02_notaciones_quimicas.ipynb)

---

📚 **[Volver al Módulo 2](README.md)** | 🏠 **[Inicio del Curso](../README.md)**

---

**Universidad de Caldas - Departamento de Química**  
*Química Computacional 173G7G*

</div>